# Aula 5: Gerando Embeddings com Modelo Encoder-Only

Este notebook demonstra como carregar um modelo encoder-only (ex: BERT) para gerar embeddings de sentenças de uma coleção de documentos. Compatível com Google Colab.

## 1. Instalar e Importar Bibliotecas Necessárias

Execute a célula abaixo para garantir que todas as bibliotecas estejam instaladas (especialmente no Colab).

> **Nota:** Os documentos separados por similaridade estão na pasta `Documentos Aula 5/` deste repositório. No Google Colab, faça o upload dos arquivos manualmente para processá-los.

In [ ]:
# Instalar bibliotecas no Colab, se necessário
import sys
if 'google.colab' in sys.modules:
    !pip install transformers torch numpy

# Importar bibliotecas essenciais
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

## 2. Carregar o Modelo Encoder-Only para Embedding

Aqui, vamos carregar um modelo encoder-only (exemplo: DistilBERT) e seu tokenizer.

In [ ]:
# Defina o nome do modelo encoder-only (pode ser alterado para outro modelo compatível)
model_name = 'distilbert-base-uncased'

# Carregar tokenizer e modelo

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Usar GPU se disponível
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

## 3. Preparar a Coleção de Documentos

Defina uma lista de sentenças ou carregue um conjunto de documentos para gerar os embeddings.

In [ ]:
# Exemplo de coleção de sentenças
documentos = [
    "O modelo BERT é muito utilizado para embeddings.",
    "Transformers revolucionaram o processamento de linguagem natural.",
    "O PyTorch é uma biblioteca popular para deep learning.",
    "Embeddings de sentenças são úteis para busca semântica.",
    "Modelos encoder-only são eficientes para tarefas de classificação."
]

# Você pode substituir a lista acima por um carregamento de arquivo/texto, se desejar.

## 4. Gerar Embeddings para Cada Sentença

Utilize o modelo e o tokenizer para gerar embeddings para cada sentença da coleção.

In [ ]:
# Função para gerar embedding de uma sentença

def gerar_embedding(sentenca, tokenizer, model, device):
    inputs = tokenizer(sentenca, return_tensors='pt', truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        # Usar o embedding do [CLS] token (primeiro token)
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy().flatten()
    return embedding

# Gerar embeddings para todas as sentenças
embeddings = np.array([
    gerar_embedding(sent, tokenizer, model, device) for sent in documentos
])

print(f"Shape dos embeddings: {embeddings.shape}")

## 5. Visualizar e Salvar os Embeddings

Visualize exemplos dos embeddings gerados e salve-os para uso posterior.

In [ ]:
# Visualizar os embeddings das primeiras sentenças
for i, sent in enumerate(documentos):
    print(f"Sentença: {sent}")
    print(f"Embedding (primeiros 5 valores): {embeddings[i][:5]}")
    print()

# Salvar os embeddings em arquivo .npy
np.save('embeddings_aula5.npy', embeddings)
print("Embeddings salvos em 'embeddings_aula5.npy'")

# Opcional: salvar em CSV
import pandas as pd
df = pd.DataFrame(embeddings, index=documentos)
df.to_csv('embeddings_aula5.csv')
print("Embeddings salvos em 'embeddings_aula5.csv'")